# Corporation Fundamentals Analysis

Yiy Zhu, Yuzhen He, Rukmal Weerawarana

FE 800 - Special Problems in Financial Engineering (MS)

Learned Sectors

---

## Notebook Setup

This section sets up core functionality required for the Jupyter notebook. This includes file loading, parsing and some basic data cleaning. It also contains source code for some helper modules used in the notebook.

This dataset is imported from the `fundamentals-data` directory, which is in the same directeory as this notebook on Google Drive.

### Package Installation

This section contains logic for installing required packages.

In [3]:
# Installing PyDrive
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 18.1MB/s 


### Jupyter Configuration

Suppress warnings.

In [0]:
import warnings

warnings.filterwarnings('ignore')

### Module Imports

This section contains module imports used by this script. The imports are grouped by their corresponding functionality for clarity.

In [0]:
# Google Drive file loading/exporting
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Formatting
from IPython.display import display, HTML

# Type hints
from typing import Callable

# Data loading
from io import StringIO
import os
import pickle

# Data manipulation
import pandas as pd

### Authentication

This section contains logic for authenticating the user with Google Drive. Forllow the prompts here for Google Drive operations to function correctly.

This functionality is adaped from the official Google Colab guide on handling external data, which can be found [here](https://colab.research.google.com/notebooks/io.ipynb).

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Data Loading

This section contains logic for file operations.

#### Loading by File ID


The function below loads the file into a serialized StringIO object, and then passes this object (treated by Python as a file) to the given data loading function. This functionality is implemented using the `PyDrive` package, whose documentation is [here](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html).

In [0]:
def loadFileByID(fileID: str) -> StringIO:
  """Function to get and load a file from Google Drive. The file is returned
  as a StringIO object, which can then be treated like a file.
  
  Arguments:
    id {str} -- Google Drive ID of the file to be downloaded. This can be found
                by turning on 'Link Sharing' in Drive, and copying the ID from
                the resulting URL.

  Returns:
    {object} -- StringIO object containing file contents from Google Drive.
  """
  
  # Getting file from Google Drive
  downloaded_file = drive.CreateFile({'id': fileID})
  
  # Serializing to StringIO object
  downloaded_file_serialized = StringIO(downloaded_file.GetContentString())

  return downloaded_file_serialized

#### Loading by File Name

The function below loads file by file name, given a folder.

In [0]:
def loadFileByName(fname: str, folderID: str):
  """Function to download files by name, given a Google Drive `folderID`. This
  function donwloads files from Drive to the local filesystem, to a file with
  name `fname`.
  
  Arguments:
    fname {str} -- Target filename (also used as save name of file).
    folderID {str} -- Google Drive ID of the folder containing the file.
  """
  # Getting files metatadata
  files_meta = drive.ListFile({'q': '\'%s\' in parents' % folderID}).GetList()

  # Building list of file names
  file_names = [i['title'] for i in files_meta]
  
  # Isolating target file; raise exception if not found
  try:
    # Solution for key-search in list of dictionaries from source below
    # See: http://bit.ly/2TETyTK
    target_file = next(i for i in files_meta if (i['title'] == fname))
  except StopIteration:
    raise FileNotFoundError('File %s not in folder %s' % (fname, folderID))
  

  # Downloading Google Drive file contents to local file
  target_file.GetContentFile(filename=fname)

#### Saving a File

The function below saves a file to disk. It verifies that a file of the same name does not already exist, and if so prompts the user for confirmation.

In [0]:
def saveToFile(fname: str, folderID: str):
  """Function to save a file to Google Drive. Assumes the file is already saved
  to the local disk with name `fname`. If a file of the same name is already
  found on Google Drive in the folder with ID `folderID`, the user is prompted
  to override. If so, the existing file is deleted, and the new one is uploaded.
  
  Arguments:
    fname {str} -- Name of the file to be saved.
    folderID {str} -- Google Drive ID of the target folder.
  """
  # Getting existing files metadata
  files_meta = drive.ListFile({'q': '\'%s\' in parents' % folderID}).GetList()
  
  # Building list of file names
  file_names = [i['title'] for i in files_meta]
  
  # Check if file with same name already exists
  if fname in file_names:
    # If it exists, verify that we are to override
    override = input('File with name %s already exists.\nOverride? [Y/n]: ' \
                     % fname)
    
    # Override not confirmed; return
    if override not in ['', 'Y', 'y', 'yes']:
      return

    # Override confirmed; deleting existing file
    
    # Solution for key-search in list of dictionaries from source below
    # See: http://bit.ly/2TETyTK
    override_file = next(i for i in files_meta if (i['title'] == fname))
    
    # Delete existing file
    override_file.Delete()
    
        
    # Log
    print('Override confirmed. Deleted file %s with Google Drive ID %s' % \
          (fname, override_file['id']))
  
  # Creating new Google Drive file, with file name `fname`
  new_file = drive.CreateFile({'title': fname,
                               'parents': [{
                                   'kind': 'drive#fileLink',
                                   'id': folderID
                               }]})

  # Adding data to the file
  try:
    new_file.SetContentFile(filename=fname)
  except FileNotFoundError:
    # File not found on disk
    print('File to be uploaded not found on disk. It must be saved locally first.')
    # Delete empty file just created
    new_file.Delete()
  
  # Upload file
  new_file.Upload()
  
  # Log
  print('Successfully uploaded file %s of size %s bytes' % \
        (fname, new_file.metadata['fileSize']))

### Google Drive File IDs

In this section, we define file IDs for each of the files we are going to be using in the script.

In [0]:
# `fundamentals_data/` folder
fid_fundamentals_data = '1m_QblIC2QH0VxSWVrBBcMtuF3hTH1jfe'

## Data Cleaning and Pre-Processing

In this section, we clean and pre-process the raw data from the `raw_data.xlsx` file in the `fundamentals_data/` folder in Google Drive.

### Data Loading

We load the raw data excel file, `raw_data.xlsx`, and the file containing our target universe.

In [11]:
# Loading raw data from Google Drive
loadFileByName(fname='raw_data.xlsx', folderID=fid_fundamentals_data)
loadFileByName(fname='universe.csv', folderID=fid_fundamentals_data)

# Loading with pandas
raw_data = pd.read_excel('raw_data.xlsx')
universe = pd.read_csv('universe.csv', header=None)[0]

# Displaying sample of raw data
raw_data.head()

,Ticker,Company Name,Unnamed: 2,SIC Code,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,A,AGILENT TECHNOLOGIES INC,NaN,3825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,AMERICAN AIRLINES GROUP INC,NaN,4512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,ADVANCE AUTO PARTS INC,NaN,5531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,APPLE INC,NaN,3571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBV,ABBVIE INC,NaN,2834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Extraction

In this section, we extract data from the (weirdly) formatted excel file.

The following is designed to extract features from the raw data Excel file. The raw data format is *very* fucked up; that's why this is messy.

The following code block contains the target features we plan to extract.

In [0]:
features = [
    'TOTAL ASSETS',
    'Cash & Equivalents',
    'Receivables - Total (Net)',
    'Inventories - Total',
    'Sales (Net)',
    'Cost of Good Sold',
    'GROSS PROFIT'
]

The following function extracts each of the features from the raw data file.

In [0]:
def getData(raw: pd.DataFrame, year: int, features: list) -> pd.DataFrame:
  # Isolating data region on spreadsheet
  data = raw[raw.iloc[:, 1] == features[0]].iloc[:, [0, year - 2006]]
  data.columns = ['Ticker', features[0]]  # Setting first column to proper ID
  
  # Setting index to be the ticker
  data.set_index('Ticker', inplace=True)
  
  # Isolating rest of the features (except the first)
  for feature in features[1:]:
    data[feature] = raw[raw.iloc[:, 1] == feature].iloc[:, year - 2006].values
  
  return data
  

We specify the year of the data we plan to extract.



In [0]:
data_year = 2010

Extracting data from the `raw_data.xlsx` file, and isolating our universe.

In [0]:
# Extracting data from raw_data.xlsx file
data = getData(raw=raw_data, year=data_year, features=features)

# Isolating universe
data = data.loc[universe]

Previewing the feature set.

In [45]:
data.head(n=20)

,TOTAL ASSETS,Cash & Equivalents,Receivables - Total (Net),Inventories - Total,Sales (Net),Cost of Good Sold,GROSS PROFIT
Ticker,,,,,,,
BBY,17849,1613,2348,5897,50272,36619,13653
SNA,3729.4,572.2,704.2,329.4,2681.5,1324.6,1356.9
RCL,19694.9,419.929,266.71,126.797,6752.5,4458.08,2294.43
EXPE,6650.99,1244.17,328.468,0,3348.11,564.43,2783.68
TRIP,722.889,113.43,51.15,0,484.635,7.345,477.29
FOXA,54384,8709,6431,2392,32778,21015,11763
AZO,5571.59,114.583,125.802,2304.58,7362.62,3458.79,3903.83
PHM,7699.38,1495.23,288.436,5174.88,4569.29,4137.94,431.345
UAA,675.378,203.87,102.034,215.355,1063.93,502.72,561.207


Saving the file to CSV and Google Drive.

In [102]:
# Saving to CSV
data.to_csv('data.csv')

# Saving to Google Drive
saveToFile(fname='data.csv', folderID=fid_fundamentals_data)

File with name data.csv already exists.
Override? [Y/n]: 
Override confirmed. Deleted file data.csv with Google Drive ID 1FUeHR--zxyozr96J0SPPi4jnENx6CqX9
Successfully uploaded file data.csv of size 19640 bytes
